<h1>Pandas I/O, práce se soubory, cestami a python argparse pro tvorbu kompletní aplikace pro automatizovanou datovou analýzu</h1>

In [1]:
import os
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

<h5>Modul pandas má vestavěné nástroje pro export a načítání velkého množství datových formátů</h5>

In [ ]:
# již známé načítání tabulek ve formátu Microsoft Excel
# vyžaduje dodatečnou instalaci modulu xlrd

df_csob = pd.read_excel(os.path.join("dataset", "csob.xls"))
df_csob

In [ ]:
# do formátu xls (Microsoft Excel) lze dataframy i zapisovat
# na ukázku vytvoříme nový dataframe z toho existujícího a uložíme na disk 
# potřebujeme modul openpyxl
csob_july = df_csob[(df_csob["date of posting"] >= '1 Jul 2022'
         ) & (df_csob["date of posting"] < '15.07.2022')]

csob_july.to_excel("./dataset/csob_july.xlsx")

In [ ]:
# čtení z textových souborů a souborů s oddělovači (CSV)
csv_sample = pd.read_csv("./dataset/sample.csv")
csv_sample

In [ ]:
# načítat soubory, například typu CSV, lze i přímo z webových zdrojů (např. http)
# zkušení CSV soubory
# https://people.sc.fsu.edu/~jburkardt/data/csv/csv.html

ford_escort = pd.read_csv('https://people.sc.fsu.edu/~jburkardt/data/csv/ford_escort.csv')
ford_escort

In [ ]:
# podobně jako při tvorbě nových dataframů lze specifikovat, které sloupce budou načteny
# parametr names specifikuje jména sloupců, header číslo řádku kde se vyskytují původní názvy sloupců ve zdrojovém souboru
ford_escort_yr_price = pd.read_csv('https://people.sc.fsu.edu/~jburkardt/data/csv/ford_escort.csv',names=["Year","Price"],header=0)
ford_escort_yr_price

In [ ]:
# podobně lze využívat i parametr usecols
biostats = pd.read_csv('https://people.sc.fsu.edu/~jburkardt/data/csv/biostats.csv', usecols=[0,2,3,4])
biostats

In [ ]:
# parameter index_col umožňuje specifikovat který ze sloupců využít k indexaci dataframu
biostats_alt_index = pd.read_csv('https://people.sc.fsu.edu/~jburkardt/data/csv/biostats.csv',index_col = "Name")
biostats_alt_index

In [16]:
# podobně jako u souborů ve formátu excel lze do CSV souborů zapisovat

biostats_alt_index.to_csv("./dataset/biostats.csv")

In [11]:
# dalšími často využívanými formáty jsou JSON, HTML, LaTeX a SQL

biostats_alt_index.to_json('./dataset/biostats.json')
biostats_alt_index.to_html('./dataset/biostats.html')
biostats_alt_index.style.to_latex('./dataset/biostats.tex') # Pouze zápis

# biostats_alt_index.to_sql() Vyžaduje nainstalovaný SQL Engine

In [ ]:
# Vizualizace HTML souboru

from IPython.display import display, HTML
with open('./dataset/biostats.html','r', encoding='utf-8') as f:  
    display(HTML(f.read()))

In [ ]:
# načtení JSON souboru modulem pandas versus načtení modulem JSON

pd.read_json('./dataset/biostats.json')

In [ ]:
import json

with open('./dataset/biostats.json', 'r', encoding='utf-8') as f:
    df_as_json = json.loads(f.read())
df_as_json

<h5>Podobně jako u tvorby dataframů z datových typů Pythonu lze při načítání specifikovat datové typy a formáty </h5>

In [ ]:
# Explicitní nastavení sloupce Age na 8bitový integer z modulu Numpy
df_stats = pd.read_csv("./dataset/biostats.csv", dtype={' "Age"':np.int8})
df_stats

In [ ]:
type(df_stats[' "Age"'][0])

In [ ]:
# Pomocí parametrů true_values a false_values lze upravit, které hodnoty mají být načteny jako Python boolean
df_truefalse = pd.read_csv('./dataset/pravda_nepravda.csv', index_col="Osoba")
df_truefalse

In [ ]:
# S parametry true_values a false_values
df_boolean = pd.read_csv('./dataset/pravda_nepravda.csv', index_col="Osoba", true_values=["Pravda"], false_values=["Nepravda"])
df_boolean

In [ ]:
type(df_boolean["Vyrok 1"][0])

<h5>Konvence a znaky využívané v načítaných souborech typu CSV</h5>


In [ ]:
# Při načítání souborů formátu CSV se mohou lišit oddělovači, kódováním znaků či znaky sloužícími k oddělenní tisíců, desetin či typek úvozovek
df_coded = pd.read_csv('./dataset/sample_coded.csv', sep=';', quotechar='"',encoding='utf-8')
df_coded

<h1>Finální cvičení - tvorba vlastního programu do příkazové řádky pro automatizované zpracování dat ve známém formátu, např. výpisu z elektronického bankovnictví</h1>

<h5>K nastavení před spuštěním bude potřeba modul argparse</h5>

<https://docs.python.org/3/library/argparse.html>

In [ ]:
#!pip install argparse
import argparse
from pathlib import Path

parser = argparse.ArgumentParser(prog="My app for data analysis",description="Takes csv/excel and processes it/computes statistics")
parser.add_argument("--src", required=True, type=Path, default=None)
parser.add_argument("--dest", required=False, type=str, default=None)
parser.add_argument("--make_figs", action=argparse.BooleanOptionalAction)
parser.add_argument("--make_summary", action=argparse.BooleanOptionalAction)

<h5>Kód výše vytvoří objekt typu argparse, nastaví které argumenty má parser očekávat</h5>

Funguje ze souborů typu .py, které se spouštějí z příkazové řádky příkazem: `python *.py [argumenty]`

<h5>Argmenty mohou být zpracovány více způsoby a mohou být předávány jako různé datové typy</h5>

<ul>
<li> type=str/int/float vrátí přímo hodnotu ve zvoleném typu 
<li> hodnoty lze specifikovat parametrem choices, který akceptuje například seznam hodnot [choice1, choice2] nebo číselný rozsah range(N)
<li> pro pouhé pravdivostní přepínače je vhodné využívat akci BooleanOptionalAction, která vrátí True pokud je mezi vstupními argumenty, jinak None (ekvivalentní s hodnotou False)
</ul>

In [ ]:
# v případě programu spouštěného z příkazové řádky bychom volali parse_args bez argumentů, které by si parser načetl z konzole/terminálu
args = parser.parse_args(["--src","./dataset/csob_july.xlsx", "--dest", "./dataset", "--make_figs"])
args

In [ ]:
args.src

V cílovém programu na zpracování dat využijte modul argparse k vytvoření několika nastavitelných možností, například výpočet variabilního množství statistik či ukládání grafů na disk

<h5>Práce se s cestami</h5>

V jazyce Python se k práci s cestami nejčastěji využívají moduly `os` a `pathlib`. Modul `os` obsahuje různé nástroje pro interakci s operačním systémem včetně cest, zatímco `pathlib` umožňuje objektově-orientovanou práci s cestami (základní objekt cesty `Path`)

In [29]:
import pathlib
import os

In [30]:
# specifikujeme složku se souboru
src = './dataset'

In [31]:
# Jako první chceme zjistit absolutní cestu k naší složce

# pomocí modulu os
os_abspath = os.path.abspath(src)
print("Absolute path with os module:",os_abspath)
# pomocí modulu pathlib
pathlib_abspath = pathlib.Path(src).absolute()
print("Absolute path with pathlib module:",pathlib_abspath)

Absolute path with os module: /Users/pberanek91/ws/Vyuka/bee it/projekty python/DA_bankovni_ucet/cviceni/tyden4/dataset
Absolute path with pathlib module: /Users/pberanek91/ws/Vyuka/bee it/projekty python/DA_bankovni_ucet/cviceni/tyden4/dataset


In [32]:
# Další krok bude vypsání obsahu složky

# modul os
os_dir_content = os.listdir(os_abspath)
print("Directory content with os module:",os_dir_content,"\n")
# modul pathlib
pathlib_dir_content = list(pathlib.Path(src).iterdir()) # vytvoří objekt typu iterable který je potřeba rozbalit do seznamu
print("Directory content with pathlib module:",pathlib_dir_content)

Directory content with os module: ['csob.xls', 'pravda_nepravda.csv', 'sample.csv', 'sample_coded.csv'] 

Directory content with pathlib module: [PosixPath('dataset/csob.xls'), PosixPath('dataset/pravda_nepravda.csv'), PosixPath('dataset/sample.csv'), PosixPath('dataset/sample_coded.csv')]
